In [1]:
!pip install tensorflow

In [2]:
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import layers, models, datasets, backend
from keras.utils import np_utils
import matplotlib.pyplot as plt
from sklearn import model_selection
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


# Dataset API 설정(train에 적용시키는 코드 못찾음)

In [3]:
#Dataset 생성 : tf.data.Dataset을 생성하는 것으로 메모리에 한번에 로드하여 사용할 수도 있으며, 동적으로 전달하여 사용할 수도 있습니다.
#Iterator 생성 : 데이터를 조회할때 사용되는 iterator 를 생성합니다.
#데이터 사용 : 실제 모델에 데이터를 입력하거나, 읽게 됩니다.

import tensorflow as tf
import numpy as np
#dataset api
train, test = tf.keras.datasets.cifar10.load_data() #데이터불러오기
x_train, y_train = train
x_test, y_test = test

dataset = tf.data.Dataset.from_tensor_slices(({"image":x_train}, y_train))#데이터셋 생성코드
dataset = dataset.shuffle(100000).repeat().batch(10) 
#위의 코드에서 생성된 Dataset 을 shuffle 함수를 이용하여 섞습니다. shuffle 함수는 고정된 버퍼 크기로 데이터를 섞는데, 
#데이터가 완전히 랜덤적으로 뒤섞기 위해서는 입력된 데이터 크기보다 큰 수를 입력해 주셔야 합니다.
#repeat라는 함수는 데이터셋을 읽다가 마지막에 도달했을 경우, 다시 처음부터 조회하는 함수입니다. 
#batch 함수는 데이터를 읽어올 개수를 지정하는 함수입니다.
iterator = dataset.make_one_shot_iterator()#one-shot iterator 생성,iterator 상태를 처음 초기화하거나 다시 초기화 하는 동작을 합니다.
next_element = iterator.get_next() #다음 항목에 연결되어 있는 tf.Tensor 객체를 리턴합니다

with tf.Session() as sess:
    print (sess.run(next_element)) #세션을 실행시켜서, 데이터를 가져와서 사용 할 수 있습니다.

W0918 14:09:53.548762  6532 deprecation.py:323] From <ipython-input-3-c65eb7868f23>:18: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


({'image': array([[[[ 80,  92, 122],
         [ 79,  90, 126],
         [ 77,  88, 124],
         ...,
         [136, 141, 168],
         [125, 129, 157],
         [147, 147, 163]],

        [[ 69,  83, 118],
         [ 71,  84, 126],
         [ 74,  86, 129],
         ...,
         [135, 140, 169],
         [124, 129, 157],
         [124, 124, 140]],

        [[ 68,  82, 120],
         [ 70,  83, 127],
         [ 72,  85, 130],
         ...,
         [133, 138, 168],
         [124, 128, 156],
         [124, 124, 140]],

        ...,

        [[ 26,  38,  66],
         [ 30,  42,  71],
         [ 30,  43,  72],
         ...,
         [ 27,  43,  73],
         [ 28,  38,  67],
         [ 83,  90, 104]],

        [[ 31,  40,  61],
         [ 37,  46,  67],
         [ 34,  44,  64],
         ...,
         [ 43,  52,  79],
         [ 45,  55,  77],
         [ 90, 100, 108]],

        [[ 97, 102, 114],
         [ 85,  89, 101],
         [ 86,  90, 102],
         ...,
         [ 79,  84,  99

In [4]:
print(type(x_train), type(y_train))
print(x_train.shape, y_train.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(50000, 32, 32, 3) (50000, 1)


In [5]:
print(x_train.shape) #형태확인
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


# 네트워크 구성하기

In [6]:
#functional api

input = Input(shape=(32,32,3))
conv1 = Conv2D(64,kernel_size=3,activation='relu',padding='same')(input)
conv2 = Conv2D(64,kernel_size=3,activation='relu',padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2,2))(conv2)
conv3 = Conv2D(128,kernel_size=3,activation='relu',padding='same')(pool1)
conv4 = Conv2D(128,kernel_size=3,activation='relu',padding='same')(conv3)
pool2 = MaxPooling2D(pool_size=(2,2))(conv4)
conv5 = Conv2D(256,kernel_size=3,activation='relu',padding='same')(pool2)
conv6 = Conv2D(256,kernel_size=3,activation='relu',padding='same')(conv5)
conv7 = Conv2D(256,kernel_size=3,activation='relu',padding='same')(conv6)
pool3 = MaxPooling2D(pool_size=(2,2))(conv7)
conv8 = Conv2D(512,kernel_size=3,activation='relu',padding='same')(pool3)
conv9 = Conv2D(512,kernel_size=3,activation='relu',padding='same')(conv8)
conv10 = Conv2D(512,kernel_size=3,activation='relu',padding='same')(conv9)
pool4= MaxPooling2D(pool_size=(2,2))(conv10)
conv11 = Conv2D(512,kernel_size=3,activation='relu',padding='same')(pool4)
conv12= Conv2D(512,kernel_size=3,activation='relu',padding='same')(conv11)
conv13 = Conv2D(512,kernel_size=3,activation='relu',padding='same')(conv12)
pool5= MaxPooling2D(pool_size=(2,2))(conv13)
flat = Flatten()(pool5)
dense1 = Dense(4096,activation='relu')(flat)
drop1 = Dropout(0.5)(dense1)
dense2 = Dense(4096,activation='relu')(drop1)
drop2 = Dropout(0.5)(dense2)
output = Dense(10,activation='softmax')(drop2)
model = Model(inputs = input,outputs = output)

W0918 14:10:22.654692  6532 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0918 14:10:22.658622  6532 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0918 14:10:22.665013  6532 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0918 14:10:22.706429  6532 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0918 14:10:22.913268  6532 deprecation_wrapper.py:119] From C:\Use

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0         
__________

In [8]:
model.compile(optimizer = 'rmsprop',
             loss = 'sparse_categorical_crossentropy',
             metrics=['acc']) # For a multi-class classification problem

W0918 14:10:23.121890  6532 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0918 14:10:23.150550  6532 deprecation_wrapper.py:119] From C:\Users\Affinity\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3341: The name tf.log is deprecated. Please use tf.math.log instead.



In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)


#기능별 정규화에 필요한 계산량
datagen.fit(x_train)

# 실시간 데이터 기능 보강을 사용하여 배치에 모델을 fit :
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=1)


W0918 14:10:26.311388  6532 deprecation.py:323] From C:\Users\Affinity\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
1563/1562 [==============================] - 2837s 2s/step - loss: 14.4996 - acc: 0.0999


# Dataset API, augmentation 없이 훈련 시키는 내용(아래)

In [144]:
#model.fit(x_train, y_train, epochs = 1, batch_size =64) 
#이 코드는 작동함 - 지난주에 했던 코드라서 끝까지 안돌림.

# 데이터증식

In [145]:
from keras.preprocessing.image import ImageDataGenerator

#augmentation
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [146]:
#데이터증식 제너레이터를 사용해서 훈련시키기
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255)

In [147]:
#파일경로 지정하기
#파일 내 test, train 이 제대로 안나눠져있고, 
#train_data는 data_batch_1~5 이런식으로 되어있음.그리고 저걸 넣으면 아래 코드가 오류남
train_dir = r'C:/Users/Affinity/cifar-10-batches-py' 
test_dir = r'C:/Users/Affinity/cifar-10-batches-py/test_batch'
batch_size = 64

In [148]:
#여기서부터 문제 안풀림
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150,150),batch_size=32, class_mode='binary')

Found 0 images belonging to 0 classes.


In [149]:
#여기도 어떻게 해결해야할지 모르겠음.
history = model.fit(train_generator, steps_per_epoch=50, epochs=100)


AttributeError: 'DirectoryIterator' object has no attribute 'ndim'

In [135]:
model.fit_generator(train_generator.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=epochs)

AttributeError: 'NumpyArrayIterator' object has no attribute 'flow'

In [131]:
train_generator = train_datagen.flow(x_train, batch_size=32)